<a href="https://colab.research.google.com/github/sagihaider/Histopathological_OralCaner_Classification/blob/main/Oral_Cancer_Resnet_F1Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# base_dir = '/content/drive/MyDrive/Colab Notebooks/Oral Cancer Classification/CancerData' # Sriramya Link
base_dir = '/content/drive/MyDrive/Data/Oral Cancer Classification/CancerData'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_resnet_v2.preprocess_input, 
                           rotation_range=180,
                           width_shift_range=0.1,
                           height_shift_range=0.1,
                           zoom_range=0.1,
                           horizontal_flip=True,
                           vertical_flip=True,
                           fill_mode='nearest',
)
image_size = 224
batch_size = 16

print("\nTrain Batches: ")
train_batches = datagen.flow_from_directory(directory=train_dir,
                                            target_size=(image_size,image_size),
                                            batch_size=batch_size,
                                            class_mode='binary',
                                            shuffle=True)

print("\nTest Batches: ")
test_batches =datagen.flow_from_directory(validation_dir,
                                           target_size=(image_size,image_size),
                                           batch_size=batch_size,                                         
                                           class_mode='binary',
                                           shuffle=False)


Train Batches: 
Found 520 images belonging to 2 classes.

Test Batches: 
Found 180 images belonging to 2 classes.


In [ ]:
#Soft Attention

from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
import tensorflow as tf

class SoftAttention(Layer):
    def __init__(self,ch,m,concat_with_x=False,aggregate=False,**kwargs):
        self.channels=int(ch)
        self.multiheads = m
        self.aggregate_channels = aggregate
        self.concat_input_with_scaled = concat_with_x
        super(SoftAttention,self).__init__(**kwargs)

    def build(self,input_shape):

        self.i_shape = input_shape

        kernel_shape_conv3d = (self.channels, 3, 3) + (1, self.multiheads) # DHWC
    
        self.out_attention_maps_shape = input_shape[0:1]+(self.multiheads,)+input_shape[1:-1]
        
        if self.aggregate_channels==False:

            self.out_features_shape = input_shape[:-1]+(input_shape[-1]+(input_shape[-1]*self.multiheads),)
        else:
            if self.concat_input_with_scaled:
                self.out_features_shape = input_shape[:-1]+(input_shape[-1]*2,)
            else:
                self.out_features_shape = input_shape
        

        self.kernel_conv3d = self.add_weight(shape=kernel_shape_conv3d,
                                        initializer='he_uniform',
                                        name='kernel_conv3d')
        self.bias_conv3d = self.add_weight(shape=(self.multiheads,),
                                      initializer='zeros',
                                      name='bias_conv3d')

        super(SoftAttention, self).build(input_shape)

    def call(self, x):

        exp_x = K.expand_dims(x,axis=-1)

        c3d = K.conv3d(exp_x,
                     kernel=self.kernel_conv3d,
                     strides=(1,1,self.i_shape[-1]), padding='same', data_format='channels_last')
        conv3d = K.bias_add(c3d,
                        self.bias_conv3d)
        conv3d = kl.Activation('relu')(conv3d)

        conv3d = K.permute_dimensions(conv3d,pattern=(0,4,1,2,3))

        
        conv3d = K.squeeze(conv3d, axis=-1)
        conv3d = K.reshape(conv3d,shape=(-1, self.multiheads ,self.i_shape[1]*self.i_shape[2]))

        softmax_alpha = K.softmax(conv3d, axis=-1) 
        softmax_alpha = kl.Reshape(target_shape=(self.multiheads, self.i_shape[1],self.i_shape[2]))(softmax_alpha)

        
        if self.aggregate_channels==False:
            exp_softmax_alpha = K.expand_dims(softmax_alpha, axis=-1)       
            exp_softmax_alpha = K.permute_dimensions(exp_softmax_alpha,pattern=(0,2,3,1,4))
   
            x_exp = K.expand_dims(x,axis=-2)
   
            u = kl.Multiply()([exp_softmax_alpha, x_exp])   
  
            u = kl.Reshape(target_shape=(self.i_shape[1],self.i_shape[2],u.shape[-1]*u.shape[-2]))(u)

        else:
            exp_softmax_alpha = K.permute_dimensions(softmax_alpha,pattern=(0,2,3,1))

            exp_softmax_alpha = K.sum(exp_softmax_alpha,axis=-1)

            exp_softmax_alpha = K.expand_dims(exp_softmax_alpha, axis=-1)

            u = kl.Multiply()([exp_softmax_alpha, x])   

        if self.concat_input_with_scaled:
            o = kl.Concatenate(axis=-1)([u,x])
        else:
            o = u
        
        return [o, softmax_alpha]

    def compute_output_shape(self, input_shape): 
        return [self.out_features_shape, self.out_attention_maps_shape]

    
    def get_config(self):
        return super(SoftAttention,self).get_config()
 

In [ ]:
from tensorflow.keras.layers import Input

MainInput=Input(shape=(224, 224, 3))

Convolution Layer1

In [ ]:
from tensorflow.keras.layers import concatenate,Dense, Conv2D, MaxPooling2D, Flatten,Input,Activation,add,AveragePooling2D,GlobalAveragePooling2D,BatchNormalization,Dropout

def convlayer1(input_value):
  conv1=Conv2D(filters=64, kernel_size=(3,3), strides=(2,2),activation="relu",padding="same")(input_value)
  conv1=BatchNormalization()(conv1)
  pool1=MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)
  return pool1

Convolution Layer2

In [ ]:
def convlayer2(input_value):
  conv2=Conv2D(filters=64, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  conv2=BatchNormalization()(conv2)
  conv2=Conv2D(filters=64, kernel_size=(3,3),activation="relu",padding="same")(conv2)
  conv2=BatchNormalization()(conv2)

  resnet=add([input_value,conv2])
  resnet=Activation("relu")(resnet)
  return resnet

Convolution Layer3

In [ ]:
def convlayer3(input_value):
  conv3=Conv2D(filters=128, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  conv3=BatchNormalization()(conv3)
  conv3=Conv2D(filters=128, kernel_size=(3,3),activation="relu",padding="same")(conv3)
  conv3=BatchNormalization()(conv3)


  skip=Conv2D(filters=128, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  skip=BatchNormalization()(skip)

  resnet=add([skip,conv3])
  resnet=Activation("relu")(resnet)
  return resnet

Convolution Layer4

In [ ]:
def convlayer4(input_value):
  conv4=Conv2D(filters=256, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  conv4=BatchNormalization()(conv4)
  conv4=Conv2D(filters=256, kernel_size=(3,3),activation="relu",padding="same")(conv4)
  conv4=BatchNormalization()(conv4)


  skip=Conv2D(filters=256, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  skip=BatchNormalization()(skip)

  resnet=add([skip,conv4])
  resnet=Activation("relu")(resnet)
  return resnet

Convolution Layer5

In [ ]:
def convlayer5(input_value):
  conv5=Conv2D(filters=512, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  conv5=BatchNormalization()(conv5)
  conv5=Conv2D(filters=512, kernel_size=(3,3),activation="relu",padding="same")(conv5)
  conv5=BatchNormalization()(conv5)


  skip=Conv2D(filters=512, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  skip=BatchNormalization()(skip)

  resnet=add([skip,conv5])
  resnet=Activation("relu")(resnet)
  return resnet

Model Creation

Block 1

In [ ]:
block1=convlayer1(MainInput)

Block 2

In [ ]:
block2=convlayer2(block1)
for i in range(0,2):
  block2=convlayer2(block2)

Block 3

In [ ]:
maxpool=MaxPooling2D(pool_size=(2,2), padding='same')(block2)
block3=convlayer3(maxpool)

for i in range(0,3):
  block3=convlayer3(block3)

Soft Attention Layer

In [ ]:
attention_layer2,map2 = SoftAttention(aggregate=True,m=16,
                                      concat_with_x=False,ch=int(block3.shape[-1]),
                                      name='soft_attention')(block3)
attention_layer2=MaxPooling2D(pool_size=(2,2), padding='same')(attention_layer2)
maxpool=MaxPooling2D(pool_size=(2,2), padding='same')(block3)

concat2=concatenate([maxpool,attention_layer2])
conv = Activation("relu")(concat2)
conv= Dropout(0.5)(conv)

Block 4

In [ ]:
block4=convlayer4(conv)
for i in range(0,5):
  block4=convlayer4(block4)

Block 5

In [ ]:
maxpool=MaxPooling2D(pool_size=(2,2), padding='same')(block4)
block5=convlayer5(maxpool)
for i in range(0,2):
  block5=convlayer5(block5)

Output Layer

In [ ]:
from tensorflow.keras import Model

output = GlobalAveragePooling2D()(block5)
output = Dense(1, activation='softmax')(output)
model = Model(inputs=MainInput, outputs=output)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 112, 112, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
from keras import backend as K
def recall_m(y_true, y_pred):

    y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), 0.5),K.floatx())
    true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.clip(y_true, 0, 1))
    recall_ratio = true_positives / (possible_positives + K.epsilon())
    return recall_ratio

def precision_m(y_true, y_pred):

    y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), 0.5), K.floatx())
    true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(y_pred)
    precision_ratio = true_positives / (predicted_positives + K.epsilon())
    return precision_ratio

def f1_m(y_true, y_pred):
    
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from keras.metrics import Precision, Recall, AUC

opt1=tf.keras.optimizers.RMSprop(learning_rate=0.01,epsilon=0.1)
model.compile(optimizer=opt1,
             loss='binary_crossentropy',
            #  metrics=['acc', 'AUC','Recall','Precision'])
            metrics = ['acc', f1_m])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

checkpoint=  ModelCheckpoint(filepath = 'ResNet34+SA.hdf5',
                             monitor='val_accuracy',
                             save_best_only=True,
                             save_weights_only=True)

In [ ]:
Earlystop = EarlyStopping(monitor='val_loss', mode='min',patience=65, min_delta=0.001)
history = model.fit(train_batches,
                    steps_per_epoch=20,
                    epochs=10,
                    verbose=2,
                    validation_data=test_batches,
                    validation_steps=8,
                    callbacks=[checkpoint,Earlystop])

Epoch 1/10
20/20 - 154s - loss: 1.2036 - acc: 0.6094 - f1_m: 0.7509 - val_loss: 329349.9062 - val_acc: 0.3750 - val_f1_m: 0.3750 - 154s/epoch - 8s/step
Epoch 2/10
20/20 - 57s - loss: 0.9432 - acc: 0.5962 - f1_m: 0.7366 - val_loss: 4515.7329 - val_acc: 0.3750 - val_f1_m: 0.3750 - 57s/epoch - 3s/step
Epoch 3/10
20/20 - 52s - loss: 0.7113 - acc: 0.6058 - f1_m: 0.7501 - val_loss: 544.2317 - val_acc: 0.3750 - val_f1_m: 0.3750 - 52s/epoch - 3s/step
Epoch 4/10
20/20 - 50s - loss: 0.6901 - acc: 0.6058 - f1_m: 0.7528 - val_loss: 20.2648 - val_acc: 0.3750 - val_f1_m: 0.3750 - 50s/epoch - 2s/step
Epoch 5/10
20/20 - 49s - loss: 0.6754 - acc: 0.6122 - f1_m: 0.7527 - val_loss: 3.1498 - val_acc: 0.3750 - val_f1_m: 0.3750 - 49s/epoch - 2s/step
Epoch 6/10
20/20 - 48s - loss: 0.6050 - acc: 0.5929 - f1_m: 0.7360 - val_loss: 5.7602 - val_acc: 0.3750 - val_f1_m: 0.3750 - 48s/epoch - 2s/step
Epoch 7/10
20/20 - 50s - loss: 0.6360 - acc: 0.6281 - f1_m: 0.7628 - val_loss: 1.4136 - val_acc: 0.3750 - val_f1_m: 0